In [4]:
from app import *

ImportError: attempted relative import with no known parent package

In [3]:
elapsed = ElapsedTime()

elapsed.t.tic()

noisy_input = channel_info.noisy_input
snr = channel_info.SNR
perfect = channel_info.perfect
number_of_pilots = channel_info.number_of_pilots
channel_model = channel_info.channel_model

train_data, train_label, val_data, val_label = channel_net_format_data()

# ------ prediction using SRCNN ------
srcnn_pred_train = SRCNN_predict(train_data, channel_model, number_of_pilots, snr)

280/280 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step


In [6]:
srcnn_pred_train.shape

(8954, 72, 14, 1)